In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__) #imprime la versión de tensorflow que usamos

In [ ]:
datos=np.loadtxt("../input/datos2D2C.txt") #este lo usamos para traer los datos

In [ ]:
print(datos)
datos.shape

In [ ]:
datos=np.transpose(datos)
x1=datos[0]
x2=datos[1]
y=datos[2]
m=len(y)
limite=int(0.8*m)
#vamos a separar los datos
x1Train, x1Test= x1[:limite],x1[limite:]
x2Train, x2Test= x2[:limite],x2[limite:]
yTrain, yTest= y[:limite],y[limite:]

In [ ]:
for i in range(len(yTrain)):
  if yTrain[i]==1:
    c='g'
  else:
    c='r'
  plt.scatter(x1Train[i],x2Train[i],color=c)
plt.show()

In [ ]:
#el coeficiente de confianza es un mejor indicador que el error 95,96
#pasos #definicion de tensores, #definición de operaciones ,#Correrlo

In [ ]:
#hiperparámetros
learning_rate=0.1
epocas=4000

In [ ]:
#definir tensores, se necesitan
X1=tf.constant(x1Train,tf.float32)
X2=tf.constant(x2Train,tf.float32)
Y=tf.constant(yTrain,tf.float32)
W=tf.Variable([0.0,0.0,0.0]) #cuando se hace redes neuronales ser llaman pesos y se utilizan w en vez T

In [ ]:
#definir operaciones
def sigmoidal(z): #definimos la función sigmoidal
    return 1/(1+np.exp(-z))

def modelo(W,X1,X2):
    return tf.sigmoid(W[0]+W[1]*X1+W[2]*X2)

def costo(W,X1,X2,Y): #recibe las mismas variables más y
    return tf.reduce_mean(-Y*tf.math.log(modelo(W,X1,X2))-(1-Y)*tf.math.log(1-modelo(W,X1,X2)))

#optimizador y logaritmo SGD
def costo_minimizar():
    return tf.reduce_mean(-Y*tf.math.log(modelo(W,X1,X2))-(1-Y)*tf.math.log(1-modelo(W,X1,X2)))

train_op=tf.keras.optimizers.SGD(learning_rate)


In [ ]:
#correr el modelo
hist_error=[]
prev_error=1000
for epoca in range(epocas):
    error=costo(W,X1,X2,Y).numpy()
    hist_error.append(error)
    train_op.minimize(costo_minimizar,var_list=[W])
    if abs (prev_error-error)<0.00001:
        print(epoca)
        break
    prev_error=error
print(W.numpy())

In [ ]:
#CALCULANDO EL COEFICIENTE DE CONFIANZA

def coef_confianza(w,x1,x2,y): #no recibe tensores
    numAciertos=0
    m=len(y)
    for i in range(m):
        y_gorrito=sigmoidal(w[0]+w[1]*x1[i]+w[2]*x2[i])
        if y_gorrito <0.5:
            clase=0
        else:
            clase=1
        if clase==y[i]:
            numAciertos+=1
    return(100*numAciertos/m)

In [ ]:
#graficamos el error
plt.plot(hist_error)
plt.show()

In [ ]:
print("Coeficiente de confianza del entrenamiento es: ")
print(coef_confianza(W.numpy(),X1.numpy(),X2.numpy(),Y.numpy()))

In [ ]:
#graficamos la sigmoidal (la frontera)
x1Frontera=[]
x2Frontera=[]

#x1F,x2F=[],[]

w=W.numpy()

for x1P in np.linspace(0,10,100): #cero a 10 son los extremos 
    for x2P in np.linspace(0,10,100): 
        yP=sigmoidal(w[0]+w[1]*x1P+w[2]*x2P)
        if abs(yP-0.5)<0.01:
            x1Frontera.append(x1P)
            x2Frontera.append(x2P)

plt.scatter(x1Frontera,x2Frontera, color="b",s=20)

#graficar los datos de nuevo
for i in range(len(yTrain)):
  if yTrain[i]==1:
    c='g'
  else:
    c='r'
  plt.scatter(x1Train[i],x2Train[i],color=c)

plt.show()
    


In [ ]:
#probarla con el test

#calcular el coeficiente de confianza con tets
print("Coeficiente de confianza del test es: ")
print(coef_confianza(W.numpy(),x1Test,x2Test,yTest))

In [ ]:
#pronósticando algunos datos
xNueva=[[2,2],[8,6],[4,2],[6,6],[5,4]] #son x1 x2
yNueva=[]
w=W.numpy()
for i in range(len(xNueva)):
    y_gorrito=sigmoidal(w[0]+w[1]*xNueva[i][0]+w[2]*xNueva[i][1])
    if y_gorrito<0.05:
        yNueva.append(0)
    else:
        yNueva.append(1)
print(yNueva)

#también se puede hacer llamando a modelo